DIMUSER

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import types

AUTOLOADER

In [0]:
df_user = spark.readStream.format("cloudfiles")\
    .option("cloudFiles.format", "parquet")\
        .option("cloudfiles.schemaLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/DimUser/checkpoint")\
        .option("schemaEvolutionMode", "addNewColumns")\
        .load("abfss://bronze@storagespotofyproject.dfs.core.windows.net/DimUser")
    

In [0]:
display(df_user)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os

import sys

project_pth = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_pth)

from utils.transformations import *


In [0]:
project_pth

In [0]:
df_user = df_user.dropDuplicates(['user_id'])


In [0]:
df_user = df_user.withColumn("user_name",upper(col("user_name")))


In [0]:
display(df_user)

In [0]:
df_user_obj = resusable()

df_user = df_user_obj.dropColumns(df_user,['_rescued_data'])

display(df_user)


In [0]:
df_user.writeStream.format("delta")\
    .outputMode("append")\
    .option("mergeSchema", "true")\
    .option("checkpointLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/DimUser/data")\
        .trigger(once=True)\
        .toTable("spotify_data.silver.DimUser")

DIM ARITIST

In [0]:
df_Art = spark.readStream.format("cloudfiles")\
    .option("cloudFiles.format", "parquet")\
        .option("cloudfiles.schemaLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/DimArtist/checkpoint")\
        .option("schemaEvolutionMode", "addNewColumns")\
        .load("abfss://bronze@storagespotofyproject.dfs.core.windows.net/DimArtist")

In [0]:
display(df_Art)


In [0]:
df_art_obj = resusable()

df_Art = df_art_obj.dropColumns(df_Art,['_rescued_data'])
df_Art = df_Art.dropDuplicates(['artist_id'])

display(df_Art)

In [0]:
df_Art.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/DimArtist/data")\
    .trigger(once=True)\
    .toTable("spotify_data.silver.DimArtist")

### DIM TRACK

In [0]:
df_track = spark.readStream.format("cloudfiles")\
    .option("cloudFiles.format", "parquet")\
        .option("cloudfiles.schemaLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/Dimtrack/checkpoint")\
        .option("schemaEvolutionMode", "addNewColumns")\
        .load("abfss://bronze@storagespotofyproject.dfs.core.windows.net/DimTrack")

In [0]:
display(df_track)

In [0]:
df_track = df_track.withColumn(
    "durationflag",
    when(col("duration_sec") < 150, "low")
    .when(col("duration_sec") > 300, "medium")
    .otherwise("high")
)

df_track = df_track.withColumn("track_name",regexp_replace(col("track_name"),'-',' '))

In [0]:
display(df_track)

In [0]:
df_track_obj = resusable()

df_track = df_track_obj.dropColumns(df_track,['_rescued_data'])
df_track = df_track.dropDuplicates(['artist_id'])

display(df_track)

In [0]:
df_track.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/Dimtrack/checkpoint")\
    .option("mergeSchema", "true")\
    .trigger(once=True)\
    .toTable("spotify_data.silver.Dimtrack")

### DIM DATE

In [0]:
df_date = spark.readStream.format("cloudfiles")\
    .option("cloudFiles.format", "parquet")\
        .option("cloudfiles.schemaLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/DimDate/checkpoint")\
        .option("schemaEvolutionMode", "addNewColumns")\
        .load("abfss://bronze@storagespotofyproject.dfs.core.windows.net/DimDate")

In [0]:
df_date_obj = resusable()

df_date= df_date_obj.dropColumns(df_date,['_rescued_data'])


display(df_track)

In [0]:
df_date.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/DimDate/checkpoint")\
    .option("mergeSchema", "true")\
    .trigger(once=True)\
    .toTable("spotify_data.silver.DimDate")

### FACT STREAM

In [0]:
df_fact = spark.readStream.format("cloudfiles")\
    .option("cloudFiles.format", "parquet")\
        .option("cloudfiles.schemaLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/FactStream/checkpoint")\
        .option("schemaEvolutionMode", "addNewColumns")\
        .load("abfss://bronze@storagespotofyproject.dfs.core.windows.net/FactStream")

In [0]:
display(df_fact)


In [0]:
df_fact_obj = resusable()

df_fact= df_fact_obj.dropColumns(df_fact,['_rescued_data'])


display(df_fact)

In [0]:
df_fact.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storagespotofyproject.dfs.core.windows.net/FactStream/checkpoint")\
    .option("mergeSchema", "true")\
    .trigger(once=True)\
    .toTable("spotify_data.silver.FactStream")